In [ ]:
{
  "cells": [
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "# RNN/LSTM pour l'analyse de sentiment\n",
        "\n",
        "##  Séance 2: Types de réseaux de neurones\n",
        "\n",
        "Ce notebook vous guidera à travers l'implémentation d'un modèle LSTM (Long Short-Term Memory) pour l'analyse de sentiment. Vous découvrirez comment les réseaux récurrents peuvent être utilisés pour comprendre et classifier du texte.\n",
        "\n",
        "### Objectifs d'apprentissage:\n",
        "- Comprendre le prétraitement du texte pour les modèles de Deep Learning\n",
        "- Découvrir l'architecture et le fonctionnement des réseaux LSTM\n",
        "- Apprendre à évaluer un modèle d'analyse de sentiment\n",
        "- Visualiser et interpréter les embeddings de mots\n",
        "\n",
        "### Prérequis:\n",
        "- Connaissances de base en Python\n",
        "- Notions fondamentales de réseaux de neurones\n",
        "- Avoir suivi la séance 1 d'introduction au Deep Learning"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 1. Configuration de l'environnement\n",
        "\n",
        "Commençons par importer les bibliothèques nécessaires et configurer notre environnement."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "import numpy as np\n",
        "import matplotlib.pyplot as plt\n",
        "import tensorflow as tf\n",
        "from tensorflow.keras.models import Sequential\n",
        "from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional\n",
        "from tensorflow.keras.preprocessing.text import Tokenizer\n",
        "from tensorflow.keras.preprocessing.sequence import pad_sequences\n",
        "from tensorflow.keras.callbacks import EarlyStopping\n",
        "import pandas as pd\n",
        "import re\n",
        "import time\n",
        "import seaborn as sns\n",
        "from sklearn.metrics import confusion_matrix, classification_report\n",
        "\n",
        "# Configuration pour reproductibilité\n",
        "np.random.seed(42)\n",
        "tf.random.set_seed(42)\n",
        "\n",
        "# Vérifier la version de TensorFlow\n",
        "print(f\"TensorFlow version: {tf.__version__}\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 2. Préparation des données\n",
        "\n",
        "Pour ce TP, nous allons utiliser un petit dataset simulé d'avis sur des films. Chaque avis sera classé comme positif, négatif ou neutre.\n",
        "\n",
        "Dans un projet réel, vous pourriez utiliser des datasets plus importants comme IMDB, Amazon Reviews, etc."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# Créer un petit jeu de données d'avis sur les films (simulé)\n",
        "reviews = [\n",
        "    \"Ce film était excellent, j'ai vraiment adoré les performances des acteurs.\",\n",
        "    \"Une expérience cinématographique incroyable, absolument à voir !\",\n",
        "    \"Un chef-d'œuvre du cinéma, magnifiquement réalisé.\",\n",
        "    \"J'ai beaucoup apprécié l'histoire et les personnages étaient bien développés.\",\n",
        "    \"Visuellement époustouflant avec une histoire captivante.\",\n",
        "    \"Un film décevant avec un scénario plein de trous.\",\n",
        "    \"Vraiment terrible, je n'ai pas aimé du tout.\",\n",
        "    \"Un gâchis complet de temps et d'argent, évitez à tout prix.\",\n",
        "    \"Ennuyeux et prévisible, les acteurs semblaient désintéressés.\",\n",
        "    \"Une déception totale, l'intrigue ne fait aucun sens.\",\n",
        "    \"C'était correct, ni bon ni mauvais.\",\n",
        "    \"Un film moyen avec quelques bons moments.\",\n",
        "    \"Certaines scènes étaient bonnes, mais dans l'ensemble assez moyen.\",\n",
        "    \"Pas aussi bon que je l'espérais, mais pas horrible non plus.\",\n",
        "    \"Une histoire intéressante mais mal exécutée.\",\n",
        "    \"Un film brillant qui m'a fait réfléchir pendant des jours.\",\n",
        "    \"Absolument sublime, l'un des meilleurs films que j'ai jamais vus.\",\n",
        "    \"Un désastre total, je me suis endormi au milieu.\",\n",
        "    \"Pas du tout ce à quoi je m'attendais, très déçu.\",\n",
        "    \"Le jeu d'acteur était fantastique, mais l'histoire était faible.\"\n",
        "]\n",
        "\n",
        "# Attribuer des sentiments (0 = négatif, 1 = neutre, 2 = positif)\n",
        "sentiments = [2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 0, 0, 1]\n",
        "\n",
        "# Convertir en DataFrame pour faciliter la manipulation\n",
        "df = pd.DataFrame({\n",
        "    'review': reviews,\n",
        "    'sentiment': sentiments\n",
        "})\n",
        "\n",
        "# Afficher quelques informations sur le dataset\n",
        "print(f\"Nombre total d'avis: {len(df)}\")\n",
        "print(f\"Répartition des sentiments: {df['sentiment'].value_counts().sort_index()}\")\n",
        "\n",
        "# Afficher quelques exemples\n",
        "print(\"\\nExemples d'avis:\")\n",
        "for sentiment in [0, 1, 2]:\n",
        "    sample = df[df['sentiment'] == sentiment].iloc[0]\n",
        "    print(f\"Sentiment {sentiment}: '{sample['review']}'\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "### Visualisation de la distribution des sentiments\n",
        "\n",
        "Vérifions que notre jeu de données est équilibré entre les différentes classes."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# Visualiser la distribution des sentiments\n",
        "plt.figure(figsize=(8, 5))\n",
        "ax = sns.countplot(x='sentiment', data=df)\n",
        "plt.title('Distribution des sentiments')\n",
        "plt.xlabel('Sentiment (0=négatif, 1=neutre, 2=positif)')\n",
        "plt.ylabel('Nombre d\\'avis')\n",
        "\n",
        "# Ajouter les valeurs sur les barres\n",
        "for p in ax.patches:\n",
        "    ax.annotate(f\"{p.get_height()}\", (p.get_x() + p.get_width()/2., p.get_height()),\n",
        "                ha='center', va='bottom')\n",
        "\n",
        "plt.show()"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 3. Prétraitement du texte\n",
        "\n",
        "Avant de pouvoir utiliser le texte avec notre modèle LSTM, nous devons le prétraiter. Cela implique plusieurs étapes:\n",
        "1. Nettoyage (minuscules, suppression de ponctuation, etc.)\n",
        "2. Tokenisation (conversion du texte en séquences de nombres)\n",
        "3. Padding (uniformisation de la longueur des séquences)\n",
        "\n",
        "Commençons par le nettoyage de texte:"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "def preprocess_text(text):\n",
        "    \"\"\"Fonction pour nettoyer et normaliser le texte\"\"\"\n",
        "    # Convertir en minuscules\n",
        "    text = text.lower()\n",
        "    # Supprimer la ponctuation et les caractères spéciaux\n",
        "    text = re.sub(r'[^\\w\\s]', '', text)\n",
        "    # Supprimer les chiffres\n",
        "    text = re.sub(r'\\d+', '', text)\n",
        "    # Supprimer les espaces multiples\n",
        "    text = re.sub(r'\\s+', ' ', text).strip()\n",
        "    return text\n",
        "\n",
        "# Appliquer le prétraitement à nos avis\n",
        "df['processed_review'] = df['review'].apply(preprocess_text)\n",
        "\n",
        "# Afficher un exemple avant et après prétraitement\n",
        "example_idx = 0\n",
        "print(f\"Avant: {df['review'][example_idx]}\")\n",
        "print(f\"Après: {df['processed_review'][example_idx]}\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "### Tokenisation du texte\n",
        "\n",
        "La tokenisation convertit le texte en séquences numériques que notre réseau de neurones peut traiter."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# Configuration pour la tokenisation\n",
        "max_words = 1000  # Taille du vocabulaire\n",
        "max_len = 100     # Longueur maximale des séquences\n",
        "\n",
        "# Créer et configurer le tokenizer\n",
        "tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')\n",
        "tokenizer.fit_on_texts(df['processed_review'])\n",
        "\n",
        "# Convertir les textes en séquences de tokens\n",
        "sequences = tokenizer.texts_to_sequences(df['processed_review'])\n",
        "\n",
        "# Appliquer le padding pour uniformiser la longueur des séquences\n",
        "padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post', truncating='post')\n",
        "\n",
        "print(f\"Taille du vocabulaire: {len(tokenizer.word_index)}\")\n",
        "print(f\"Forme des séquences après padding: {padded_sequences.shape}\")\n",
        "\n",
        "# Afficher le mapping de quelques mots vers leurs tokens\n",
        "print(\"\\nExemples de mapping mot -> token:\")\n",
        "sample_words = ['film', 'bon', 'mauvais', 'excellent', 'terrible']\n",
        "for word in sample_words:\n",
        "    if word in tokenizer.word_index:\n",
        "        print(f\"{word} -> {tokenizer.word_index[word]}\")\n",
        "    else:\n",
        "        print(f\"{word} -> Non trouvé dans le vocabulaire\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "### Visualisation d'une séquence tokenisée\n",
        "\n",
        "Pour mieux comprendre la tokenisation, visualisons comment un avis est converti en séquence de tokens."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "def visualize_tokenized_sequence(text, tokens):\n",
        "    \"\"\"Visualise la correspondance entre mots et tokens\"\"\"\n",
        "    words = text.split()\n",
        "    plt.figure(figsize=(15, 3))\n",
        "    plt.bar(range(len(tokens)), tokens)\n",
        "    plt.xticks(range(len(tokens)), words, rotation=45, ha='right')\n",
        "    plt.ylabel('Token ID')\n",
        "    plt.title('Représentation tokenisée d\\'un avis')\n",
        "    plt.tight_layout()\n",
        "    plt.show()\n",
        "\n",
        "sample_idx = 0\n",
        "sample_text = df['processed_review'][sample_idx].split()[:15]  # Limiter à 15 mots pour lisibilité\n",
        "sample_tokens = sequences[sample_idx][:15]\n",
        "\n",
        "print(f\"Exemple d'avis: {' '.join(sample_text)}\")\n",
        "print(f\"Tokens correspondants: {sample_tokens}\")\n",
        "visualize_tokenized_sequence(' '.join(sample_text), sample_tokens)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 4. Division en ensembles d'entraînement et de test\n",
        "\n",
        "Avant de créer notre modèle, divisons nos données en ensembles d'entraînement et de test pour évaluer ses performances."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "from sklearn.model_selection import train_test_split\n",
        "\n",
        "# Division 70-30 avec stratification pour conserver la distribution des classes\n",
        "X_train, X_test, y_train, y_test = train_test_split(\n",
        "    padded_sequences, \n",
        "    df['sentiment'],\n",
        "    test_size=0.3,\n",
        "    random_state=42,\n",
        "    stratify=df['sentiment']  # Assurer une répartition équilibrée des classes\n",
        ")\n",
        "\n",
        "print(f\"Forme des données d'entraînement: {X_train.shape}\")\n",
        "print(f\"Forme des données de test: {X_test.shape}\")\n",
        "print(f\"Distribution des classes (entraînement): {pd.Series(y_train).value_counts().sort_index()}\")\n",
        "print(f\"Distribution des classes (test): {pd.Series(y_test).value_counts().sort_index()}\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 5. Création du modèle LSTM\n",
        "\n",
        "Nous allons maintenant créer notre modèle d'analyse de sentiment en utilisant une architecture LSTM bidirectionnelle.\n",
        "\n",
        "### Architecture du modèle\n",
        "- **Couche d'embedding**: Convertit les tokens en vecteurs denses\n",
        "- **Couches LSTM bidirectionnelles**: Capture les dépendances à long terme dans les deux directions\n",
        "- **Dropout**: Évite le surapprentissage\n",
        "- **Couche dense finale**: Classification en 3 catégories (négatif, neutre, positif)"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# Paramètres du modèle\n",
        "embedding_dim = 32  # Dimension de l'espace d'embedding\n",
        "\n",
        "# Création du modèle\n",
        "model = Sequential([\n",
        "    # Couche d'embedding pour convertir les tokens en vecteurs denses\n",
        "    Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len),\n",
        "    \n",
        "    # Couche LSTM bidirectionnelle\n",
        "    Bidirectional(LSTM(64, return_sequences=True)),\n",
        "    \n",
        "    # Deuxième couche LSTM suivie de dropout pour régularisation\n",
        "    Bidirectional(LSTM(32)),\n",
        "    Dropout(0.5),\n",
        "    \n",
        "    # Couche de classification (3 classes: négatif, neutre, positif)\n",
        "    Dense(3, activation='softmax')\n",
        "])\n",
        "\n",
        "# Afficher un résumé du modèle\n",
        "model.summary()"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "### 💡 Points clés à observer dans l'architecture\n",
        "\n",
        "- **LSTM bidirectionnel** : Lit le texte de gauche à droite ET de droite à gauche, capturant mieux le contexte\n",
        "- **return_sequences=True** : Permet d'empiler plusieurs couches LSTM\n",
        "- **Dropout** : Désactive aléatoirement 50% des neurones pendant l'entraînement pour éviter le surapprentissage\n",
        "- **Activation softmax** : Génère une distribution de probabilité sur les 3 classes"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 6. Compilation et entraînement du modèle\n",
        "\n",
        "Maintenant, compilons et entraînons notre modèle LSTM."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# Compiler le modèle\n",
        "model.compile(\n",
        "    optimizer='adam',\n",
        "    loss='sparse_categorical_crossentropy',  # Pour les étiquettes sous forme d'entiers (non one-hot)\n",
        "    metrics=['accuracy']\n",
        ")\n",
        "\n",
        "# Early stopping pour éviter le surapprentissage\n",
        "early_stopping = EarlyStopping(\n",
        "    monitor='val_loss',\n",
        "    patience=3,\n",
        "    restore_best_weights=True\n",
        ")\n",
        "\n",
        "# Mesure du temps d'entraînement\n",
        "start_time = time.time()\n",
        "\n",
        "# Entraînement du modèle\n",
        "history = model.fit(\n",
        "    X_train, \n",
        "    y_train, \n",
        "    epochs=20,\n",
        "    batch_size=4,  # Petit batch size en raison de la petite taille du dataset\n",
        "    validation_split=0.2,  # 20% des données d'entraînement serviront à la validation\n",
        "    callbacks=[early_stopping],\n",
        "    verbose=1\n",
        ")\n",
        "\n",
        "training_time = time.time() - start_time\n",
        "print(f\"\\nTemps d'entraînement: {training_time:.2f} secondes\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "### Visualisation de l'évolution de l'entraînement\n",
        "\n",
        "Observons comment la précision et la perte ont évolué au cours de l'entraînement."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# Visualisation de l'entraînement\n",
        "plt.figure(figsize=(12, 5))\n",
        "\n",
        "# Graphique de précision\n",
        "plt.subplot(1, 2, 1)\n",
        "plt.plot(history.history['accuracy'], label='Entraînement')\n",
        "plt.plot(history.history['val_accuracy'], label='Validation')\n",
        "plt.title('Évolution de la précision')\n",
        "plt.xlabel('Époque')\n",
        "plt.ylabel('Précision')\n",
        "plt.legend()\n",
        "plt.grid(True, linestyle='--', alpha=0.6)\n",
        "\n",
        "# Graphique de perte\n",
        "plt.subplot(1, 2, 2)\n",
        "plt.plot(history.history['loss'], label='Entraînement')\n",
        "plt.plot(history.history['val_loss'], label='Validation')\n",
        "plt.title('Évolution de la perte')\n",
        "plt.xlabel('Époque')\n",
        "plt.ylabel('Perte')\n",
        "plt.legend()\n",
        "plt.grid(True, linestyle='--', alpha=0.6)\n",
        "\n",
        "plt.tight_layout()\n",
        "plt.show()"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 7. Évaluation du modèle\n",
        "\n",
        "Maintenant, évaluons les performances de notre modèle sur l'ensemble de test."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# Évaluation sur l'ensemble de test\n",
        "test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)\n",
        "print(f\"Précision sur l'ensemble de test: {test_acc*100:.2f}%\")\n",
        "\n",
        "# Générer les prédictions\n",
        "y_pred_proba = model.predict(X_test)\n",
        "y_pred_classes = np.argmax(y_pred_proba, axis=1)\n",
        "\n",
        "# Matrice de confusion\n",
        "conf_mat = confusion_matrix(y_test, y_pred_classes)\n",
        "plt.figure(figsize=(8, 6))\n",
        "sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues', \n",
        "            xticklabels=['Négatif', 'Neutre', 'Positif'],\n",
        "            yticklabels=['Négatif', 'Neutre', 'Positif'])\n",
        "plt.xlabel('Prédit')\n",
        "plt.ylabel('Réel')\n",
        "plt.title('Matrice de confusion')\n",
        "plt.tight_layout()\n",
        "plt.show()\n",
        "\n",
        "# Rapport de classification\n",
        "print(\"\\nRapport de classification détaillé:\")\n",
        "target_names = ['Négatif', 'Neutre', 'Positif']\n",
        "print(classification_report(y_test, y_pred_classes, target_names=target_names))"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "### 🧠 Réflexions sur les résultats\n",
        "\n",
        "- **Analysez la matrice de confusion**: Quelles classes sont le mieux reconnues? Y a-t-il des confusions particulières?\n",
        "- **Précision vs Rappel**: Y a-t-il un déséquilibre? Quelle métrique privilégier selon le contexte?\n",
        "- **Taille du dataset**: Comment les résultats pourraient-ils être affectés par la petite taille de notre jeu de données?\n",
        "\n",
        "👉 **Discussion**: Notez vos observations ci-dessous:"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "*Écrivez vos observations ici...*"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 8. Test avec de nouveaux avis\n",
        "\n",
        "Testons maintenant notre modèle avec quelques nouveaux avis qui n'ont pas été utilisés pour l'entraînement."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# Nouveaux avis à tester\n",
        "new_reviews = [\n",
        "    \"Ce film était vraiment fantastique, j'ai adoré chaque minute.\",\n",
        "    \"Je n'ai pas du tout aimé ce film, c'était une perte de temps complète.\",\n",
        "    \"C'était un film correct, ni bon ni mauvais.\"\n",
        "]\n",
        "\n",
        "# Prétraitement des nouveaux avis\n",
        "processed_new_reviews = [preprocess_text(review) for review in new_reviews]\n",
        "sequences_new = tokenizer.texts_to_sequences(processed_new_reviews)\n",
        "padded_new = pad_sequences(sequences_new, maxlen=max_len, padding='post', truncating='post')\n",
        "\n",
        "# Prédictions\n",
        "predictions = model.predict(padded_new)\n",
        "predicted_classes = np.argmax(predictions, axis=1)\n",
        "\n",
        "# Afficher les résultats\n",
        "sentiment_labels = {0: \"Négatif\", 1: \"Neutre\", 2: \"Positif\"}\n",
        "\n",
        "print(\"Prédictions pour les nouveaux avis:\\n\")\n",
        "for i, review in enumerate(new_reviews):\n",
        "    pred_class = predicted_classes[i]\n",
        "    confidence = predictions[i][pred_class] * 100\n",
        "    \n",
        "    print(f\"Avis: {review}\")\n",
        "    print(f\"Sentiment prédit: {sentiment_labels[pred_class]} (confiance: {confidence:.2f}%)\")\n",
        "    print(\"Probabilités pour chaque classe:\")\n",
        "    for j, label in sentiment_labels.items():\n",
        "        print(f\"  {label}: {predictions[i][j]*100:.2f}%\")\n",
        "    print()"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "### Visualisation graphique des prédictions\n",
        "\n",
        "Visualisons les probabilités pour chaque classe pour les nouveaux avis."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# Visualisation des probabilités pour chaque avis\n",
        "plt.figure(figsize=(15, 5))\n",
        "labels = ['Négatif', 'Neutre', 'Positif']\n",
        "\n",
        "for i, review in enumerate(new_reviews):\n",
        "    plt.subplot(1, 3, i+1)\n",
        "    plt.bar(labels, predictions[i], color=['red', 'gray', 'green'])\n",
        "    plt.title(f\"Avis {i+1}\")\n",
        "    plt.ylim(0, 1)\n",
        "    plt.ylabel('Probabilité')\n",
        "    plt.xticks(rotation=45)\n",
        "    \n",
        "    # Ajouter les valeurs sur les barres\n",
        "    for j, p in enumerate(predictions[i]):\n",
        "        plt.text(j, p + 0.02, f\"{p*100:.1f}%\", ha='center')\n",
        "        \n",
        "plt.tight_layout()\n",
        "plt.show()"
      ]
    },